In [61]:
def set_media_conditions(model_file, biomass_ID, media_file, experimental_growth): 
    # Including the versions I used when developing -->
    import pandas as pd #Version 2.2.3
    import cobra #Version 0.29.0
    from cobra.io import read_sbml_model
    '''
    GOAL: Define a dictionary for the media conditions on which a cell (line) to be modelled has been grown
    
    PROCESS: 
        1. Upload a CSV containing corresponding model reactions, metabolite concentrations, corresponding reactions and reaction boundaries
        2. Iterate through the model, constraining each exchange reaction one-by-one
        2a. If the exchange reaction is in the defined CSV, lower bound (uptake) will be set to the experimental g/L
        2b. If the exchange reaction is not in the defined CSV, lower bound (uptake) is constrained to zero
        2c. Each time an exchange reaction is constrained, FBA maximised for growth is solved and objective value is compared to experimental growth
        2d. If predicted growth is slower than experimental, exchange is reopened

    LOGIC AND ASSUMMPTIONS:
        - The maximum concentration of a metabolite which the cell can consume is equal to the concentration in the media
        - g/L equates to mmol/gDW/hour uptake
        - If the cell cannot grow without this metabolite, but we have not defined it, it must be in some undefined media components, e.g. serum
        - The cell is maximising biomass production
    
    PARAMETERS:
        - 'model_file': A COBRA model, not yet constrained with 'omics data, I use XML file but might work with another file type
        - 'biomass_ID' is the ID for the biomass equation in the model you are using
        - 'media_file': a CSV containing the media definition, see example 'DMEM.csv' in folder
        - Column 1: metabolite name, column 2: mg/L, column 3: g/L, column 4: reaction ID, column 5: lower_bound, column 6: upper bound
        - 'experimental_growth': the experimental groth rate of the cell, to calculate this take the inverse of the cell's doubline time, e.g. 1/20 hours == 0.05 g/gDW/hour 
    
    Returns:
        - Dictionary
        - Predicted growth with media constraints, FBA solved for growth maximisation
        - List of reopened reactions (assumed to be contained in undefined media component)
        - List of defined reactions (pre-specified in the 'media_file' CSV file
        - List of closed reactions (assumed not to be consumed by the cell in the state being modelled)

    Return types:
        - Dictionary
        - Float
        - List
        - List
        - List
    '''
    # Load model and store exchange reactions as list
    model = read_sbml_model(model_file)
    exchanges = []
    for e in model.exchanges:
        exchanges.append(e.id)
    
    # Import defined media conditions
    media_csv = pd.read_csv(media_file)

    # Find the essential reactions, as these can't be constrained
    print('Finding essential reactions...')
    model.objective = model.reactions.get_by_id(biomass_ID)
    essential_reactions = cobra.flux_analysis.variability.find_essential_reactions(model)
    essential_list = []
    for r in essential_reactions:
        essential_list.append(r.id)
    
    # Make an initial media dictionary, which will then be iterated through to find reactions that need reopening
    media = {}
    for n in range(len(media_csv)):
        if media_csv.iloc[n,3] in exchanges:
            if media_csv.iloc[n,3] not in essential_list:
                media[media_csv.iloc[n,3]] = (media_csv.iloc[n,4],media_csv.iloc[n,5])

    # Go through exchange reactions and add constraints
    for e in exchanges:
        if e not in essential_list: # don't want to constrain essential reactions or cell won't grow
            if e not in media.keys(): # don't want to constrain lower bound to zero on reactions that we have a concentration for
                media[e] = (0,1000) # add exchange reactions one-by-one into the media dictionary
                model.medium = media #this adds all the defined constraints as well as these one-by-one closings of exchanges
                print(e, 'closed growth:', model.slim_optimize())
                if model.slim_optimize() <= 0:
                    media[e] = (-1000,1000)
                    model.medium = media
                    print(e, 'reopened growth:', model.slim_optimize())
                elif model.slim_optimize() < experimental_growth:
                    media[e] = (-1000,1000)
                    model.medium = media
                    print(e, 'reopened growth:', model.slim_optimize())
                else:
                    continue
    final_media = media

    # Make string for the objective value with final dictionary added
    model.medium = final_media
    final_growth = model.slim_optimize()
    doubling_time = 1/final_growth
    print('Final growth with media constraints:', final_growth, 'g/gDW/hour', '==', doubling_time, 'hours')

    # Categorise final boundaries of the media and print results
    reopened_reactions = []
    closed_reactions = []
    defined_reactions = []
    for k,v in media.items():
        if v == (-1000,1000):
            reopened_reactions.append(k)
        if v == (0,1000):
            closed_reactions.append(k)
        if v != (0,1000):
            if v != (-1000,1000):
                defined_reactions.append(k)
    print('Defined reactions:', len(defined_reactions))
    print('Closed reactions:', len(closed_reactions))
    print('Reopened reactions:', len(reopened_reactions))
    print('Number of exchange reactions:',len(model.exchanges))
    reopened_metabolites = []
    for r in reopened_reactions:
        for m in model.reactions.get_by_id(r.id):
            reopened_metabolites.append(m.name)
    print('Metabolites in reopened reactions:',reopened_metabolites)
    defined_metabolites = []
    for r in defined_reactions:
        for m in model.reactions.get_by_id(r.id):
            defined_metabolites.append(m.name)
    print('Metabolites in defined reactions:',defined_metabolites)
    
    return(final_media, final_growth, reopened_reactions, defined_reactions, closed_reactions)

In [90]:
def set_media_conditions_1(model_file, essential_reactions_list_name, biomass_ID, media_file, experimental_growth): 
    # Including the versions I used when developing -->
    import pandas as pd #Version 2.2.3
    import cobra #Version 0.29.0
    from cobra.io import read_sbml_model
    '''
    GOAL: Define a dictionary for the media conditions on which a cell (line) to be modelled has been grown
    
    PROCESS: 
        1. Upload a CSV containing corresponding model reactions, metabolite concentrations, corresponding reactions and reaction boundaries
        2. Iterate through the model, constraining each exchange reaction one-by-one
        2a. If the exchange reaction is in the defined CSV, lower bound (uptake) will be set to the experimental g/L
        2b. If the exchange reaction is not in the defined CSV, lower bound (uptake) is constrained to zero
        2c. Each time an exchange reaction is constrained, FBA maximised for growth is solved and objective value is compared to experimental growth
        2d. If predicted growth is slower than experimental, exchange is reopened

    LOGIC AND ASSUMMPTIONS:
        - The maximum concentration of a metabolite which the cell can consume is equal to the concentration in the media
        - g/L equates to mmol/gDW/hour uptake
        - If the cell cannot grow without this metabolite, but we have not defined it, it must be in some undefined media components, e.g. serum
        - The cell is maximising biomass production
    
    PARAMETERS:
        - 'model_file': A COBRA model, not yet constrained with 'omics data, I use XML file but might work with another file type
        - 'biomass_ID' is the ID for the biomass equation in the model you are using
        - 'media_file': a CSV containing the media definition, see example 'DMEM.csv' in folder
        - Column 1: metabolite name, column 2: mg/L, column 3: g/L, column 4: reaction ID, column 5: lower_bound, column 6: upper bound
        - 'experimental_growth': the experimental groth rate of the cell, to calculate this take the inverse of the cell's doubline time, e.g. 1/20 hours == 0.05 g/gDW/hour 
    
    Returns:
        - Dictionary
        - Predicted growth with media constraints, FBA solved for growth maximisation
        - List of reopened reactions (assumed to be contained in undefined media component)
        - List of defined reactions (pre-specified in the 'media_file' CSV file
        - List of closed reactions (assumed not to be consumed by the cell in the state being modelled)

    Return types:
        - Dictionary
        - Float
        - List
        - List
        - List
    '''
    # Load model and store exchange reactions as list
    model = read_sbml_model(model_file)
    exchanges = []
    for e in model.exchanges:
        exchanges.append(e.id)

    # Solve original growth on unconstrained model
    solution = model.optimize()
    original_growth = float(solution.fluxes[biomass_ID])
    print('Original growth',original_growth)
    
    # Import defined media conditions
    media_csv = pd.read_csv(media_file)

    # Find the essential reactions, as these can't be constrained
    print('Finding essential reactions...')
    essential_list = essential_reactions_list_name
    
    # Make an initial media dictionary, which will then be iterated through to find reactions that need reopening
    media = {}
    for n in range(len(media_csv)):
        if media_csv.iloc[n,3] in exchanges:
            if media_csv.iloc[n,3] not in essential_list:
                media[media_csv.iloc[n,3]] = float(media_csv.iloc[n,4])

    # Go through exchange reactions and add constraints
    for e in exchanges:
        if e not in essential_list: # don't want to constrain essential reactions or cell won't grow
            if e not in media.keys(): # don't want to constrain lower bound to zero on reactions that we have a concentration for
                media[e] = 0 # add exchange reactions one-by-one into the media dictionary
                model.medium = media #this adds all the defined constraints as well as these one-by-one closings of exchanges
                solution = model.optimize()
                growth = float(solution.fluxes[biomass_ID])
                print(e, 'closed growth:', growth)
                if growth <= 0:
                    media[e] = -1000
                    model.medium = media
                    solution = model.optimize()
                    growth = float(solution.fluxes[biomass_ID])
                    print(e, 'reopened growth:', growth)
                elif growth < experimental_growth:
                    media[e] = -1000
                    model.medium = media
                    solution = model.optimize()
                    growth = float(solution.fluxes[biomass_ID])
                    print(e, 'reopened growth:', growth)
                else:
                    continue
    final_media = media

    # Make string for the objective value with final dictionary added
    model.medium = final_media
    solution = model.optimize()
    final_growth = float(solution.fluxes[biomass_ID])
    doubling_time = 1/final_growth
    print('Final growth with media constraints:', final_growth, 'g/gDW/hour', '==', doubling_time, 'hours')

    # Categorise final boundaries of the media and print results
    reopened_reactions = []
    closed_reactions = []
    defined_reactions = []
    for k,v in media.items():
        if v == (-1000,1000):
            reopened_reactions.append(k)
        if v == (0,1000):
            closed_reactions.append(k)
        if v != (0,1000):
            if v != (-1000,1000):
                defined_reactions.append(k)
    print('Defined reactions:', len(defined_reactions))
    print('Closed reactions:', len(closed_reactions))
    print('Reopened reactions:', len(reopened_reactions))
    print('Number of exchange reactions:',len(model.exchanges))
    reopened_metabolites = []
    for r in reopened_reactions:
        for m in model.reactions.get_by_id(r.id):
            reopened_metabolites.append(m.name)
    print('Metabolites in reopened reactions:',reopened_metabolites)
    defined_metabolites = []
    for r in defined_reactions:
        for m in model.reactions.get_by_id(r.id):
            defined_metabolites.append(m.name)
    print('Metabolites in defined reactions:',defined_metabolites)
    
    return(final_media, final_growth, reopened_reactions, defined_reactions, closed_reactions)

In [59]:
essential_list = []
for r in essential_reactions:
    essential_list.append(r.id)

In [81]:
exchanges = []
for e in model.exchanges:
    exchanges.append(e.id)
media = {}
for n in range(len(media_csv)):
    if media_csv.iloc[n,3] in exchanges:
        if media_csv.iloc[n,3] not in essential_list:
            media[media_csv.iloc[n,3]] = float(media_csv.iloc[n,4])

In [82]:
media

{'MAR09066': -0.084,
 'MAR09065': -0.063,
 'MAR09063': -0.584,
 'MAR09067': -0.03,
 'MAR09038': -0.042,
 'MAR09039': -0.105,
 'MAR09040': -0.105,
 'MAR09041': -0.146,
 'MAR09042': -0.03,
 'MAR09043': -0.066,
 'MAR09069': -0.042,
 'MAR09044': -0.095,
 'MAR09045': -0.016,
 'MAR09064': -0.104,
 'MAR09046': -0.094,
 'MAR09083': -0.004,
 'MAR09146': -0.004,
 'MAR09361': -0.0072,
 'MAR09378': -0.004,
 'MAR09145': -0.004,
 'MAR09144': -0.004,
 'MAR09143': -0.0004,
 'MAR09159': -0.004,
 'MAR09034': -4.5}

In [86]:
essential_list.append('MAR07108')

In [91]:
result = set_media_conditions_1('Human-GEM.xml', essential_list, 'MAR13082', 'DMEM.csv', 0.038)
result
#it's just something weird with the essential_reactions function, maybe I need to define this myself and compare lists...
#save original constraints of each reaction in loop and add them back if closed growth is zero
#or instead of using the model.medium function just change the lower bounds myself

Original growth 187.35362997658086
Finding essential reactions...


C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


MAR07110 closed growth: 0.0
MAR07110 reopened growth: 0.0


C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


MAR07112 closed growth: 0.0
MAR07112 reopened growth: 0.0
MAR07114 closed growth: 0.0


...

C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


MAR07126 reopened growth: 0.0


C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\p04121km\AppData\Local\anaconda3\envs\prosperity_project\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


MAR09023 closed growth: 0.0


KeyboardInterrupt: 

In [ ]:
#change upper import bands (0,1000) to just 0 and so on

In [83]:
media = {}
model.medium = media
solution = model.optimize()
growth = solution.fluxes['MAR13082']
float(growth)

8.644917859936877e-15

In [ ]:
#make a modified version that doesn't need to solve essential reactions then save the non modified version

In [42]:
essential_reactions = cobra.flux_analysis.variability.find_essential_reactions(model)
essential_reactions

{<Reaction MAR00578 at 0x1b2a2de60d0>,
 <Reaction MAR00579 at 0x1b2a0fa3990>,
 <Reaction MAR00580 at 0x1b2a0fa8a50>,
 <Reaction MAR00581 at 0x1b2a0fa9f50>,
 <Reaction MAR00598 at 0x1b2a0fcdf10>,
 <Reaction MAR02129 at 0x1b2a1b49490>,
 <Reaction MAR02131 at 0x1b2a1b4b010>,
 <Reaction MAR02133 at 0x1b2a1b55110>,
 <Reaction MAR02134 at 0x1b2a1b54f90>,
 <Reaction MAR02135 at 0x1b2a1b56bd0>,
 <Reaction MAR02136 at 0x1b2a1b57ad0>,
 <Reaction MAR02139 at 0x1b2a1b54d90>,
 <Reaction MAR02142 at 0x1b2a1b5f290>,
 <Reaction MAR02143 at 0x1b2a1b6c190>,
 <Reaction MAR02144 at 0x1b2a1b6cf50>,
 <Reaction MAR02145 at 0x1b2a1b6de10>,
 <Reaction MAR04020 at 0x1b29ed66b90>,
 <Reaction MAR04030 at 0x1b29ee6c6d0>,
 <Reaction MAR04204 at 0x1b2a1b139d0>,
 <Reaction MAR04269 at 0x1b2a17a6d90>,
 <Reaction MAR04371 at 0x1b29eb63bd0>,
 <Reaction MAR05130 at 0x1b29f62aa50>,
 <Reaction MAR05131 at 0x1b29f634410>,
 <Reaction MAR05132 at 0x1b29f634f50>,
 <Reaction MAR05133 at 0x1b29f635a90>,
 <Reaction MAR05134 at 0x

In [ ]:
#DMEM comes from https://www.thermofisher.com/order/catalog/product/11965092?SID=srch-srp-11965092
#experimental growth that I have used is same as in my thesis which was 0.038 g/gDW/hour